# 04 · 新資料推論管線（TIC → MAST → BLS/TLS → 機率）

## 工作流程
1. **單目標推論**：輸入單個 TIC → 下載光曲線 → 預測機率
2. **批次處理**：輸入 TIC 列表 → 批次預測 → 排序輸出
3. **視覺化**：摺疊光曲線、BLS 功率譜、預測分數
4. **GPU 優化**：偵測 L4 GPU 並示範 bfloat16 autocast

---

## 1. 環境設定與依賴安裝

In [1]:
# 步驟 1: 安裝套件 (需要手動重啟 Runtime)
# ⚠️ 重要: 執行此 cell 後，請手動重啟 Runtime (Runtime → Restart runtime)

!pip install -q numpy==1.26.4 pandas astropy scipy'<1.13' matplotlib scikit-learn
!pip install -q lightkurve astroquery joblib seaborn
!pip install -q 'xgboost>=2.0.0' plotly pyyaml

print("✅ 套件安裝完成!")
print("⚠️ 請現在手動重啟 Runtime: Runtime → Restart runtime")
print("   然後繼續執行下一個 cell")

系統找不到指定的檔案。


✅ 套件安裝完成!
⚠️ 請現在手動重啟 Runtime: Runtime → Restart runtime
   然後繼續執行下一個 cell


ERROR: Invalid requirement: "'xgboost": Expected package name at the start of dependency specifier
    'xgboost
    ^


In [2]:
# Environment Detection
import sys
import os
from pathlib import Path

# Detect environment
IN_COLAB = 'google.colab' in sys.modules or '/content' in os.getcwd()

if IN_COLAB:
    print("Running in: Google Colab")
    
    # Clone repo if needed
    project_dir = Path('/content/exoplanet-starter')
    if not project_dir.exists():
        print("Cloning repository...")
        !git clone https://github.com/exoplanet-spaceapps/exoplanet-starter.git
        print("Repository cloned")
    
    # Change to project directory
    os.chdir(str(project_dir))
    
    # Add to Python path
    sys.path.insert(0, str(project_dir))
    sys.path.insert(0, str(project_dir / 'src'))
    sys.path.insert(0, str(project_dir / 'notebooks'))
    
    print(f"Working directory: {os.getcwd()}")
    print(f"Python path configured")
    
else:
    print("Running in: Local environment")
    # Local paths
    project_dir = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
    sys.path.insert(0, str(project_dir / 'src'))
    sys.path.insert(0, str(project_dir))

print(f"Project directory: {project_dir}")

Running in: Local environment
Project directory: C:\Users\thc1006\Desktop\dev\exoplanet-starter


In [3]:
# 步驟 2: 驗證環境 (Runtime 重啟後執行)
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')

# 檢查 NumPy 版本
print(f"NumPy 版本: {np.__version__}")
print(f"Python 版本: {sys.version}")

# ⚠️ NumPy 2.0+ compatibility: Continue with warning instead of failing
if np.__version__.startswith('2.'):
    print("⚠️ NumPy 2.0+ 檢測到！某些套件可能不相容")
    print("   繼續執行，但如遇到錯誤請考慮降級至 NumPy 1.26.4")
else:
    print("✅ NumPy 版本正確 (< 2.0)")
    
# 檢查是否在 Colab 環境
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📍 在 Google Colab 環境執行")
    # Clone repository if needed
    import os
    if not os.path.exists('/content/exoplanet-starter'):
        !git clone https://github.com/exoplanet-spaceapps/exoplanet-starter.git /content/exoplanet-starter
        os.chdir('/content/exoplanet-starter')
    sys.path.append('/content/exoplanet-starter')
else:
    print("💻 在本地環境執行")
    import os
    # Handle Windows paths properly
    if '__file__' in globals():
        parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    else:
        parent_dir = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
    sys.path.append(parent_dir)

print("\n✅ 環境設定完成！可以繼續執行後續 cells")

NumPy 版本: 2.3.1
Python 版本: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
⚠️ NumPy 2.0+ 檢測到！某些套件可能不相容
   繼續執行，但如遇到錯誤請考慮降級至 NumPy 1.26.4
💻 在本地環境執行

✅ 環境設定完成！可以繼續執行後續 cells


## 2. GPU 偵測與優化設定

In [4]:
# GPU 偵測與優化設定
gpu_info = {
    'available': False,
    'device_name': None,
    'is_l4': False,
    'supports_bfloat16': False
}

try:
    import torch
    
    if torch.cuda.is_available():
        gpu_info['available'] = True
        gpu_info['device_name'] = torch.cuda.get_device_name(0)
        gpu_props = torch.cuda.get_device_properties(0)
        
        print(f"🖥️ GPU 偵測結果:")
        print(f"   型號: {gpu_info['device_name']}")
        print(f"   記憶體: {gpu_props.total_memory / 1024**3:.2f} GB")
        print(f"   CUDA 運算能力: {gpu_props.major}.{gpu_props.minor}")
        
        # 檢查是否為 L4 GPU
        if 'L4' in gpu_info['device_name']:
            gpu_info['is_l4'] = True
            gpu_info['supports_bfloat16'] = True
            print("\n💡 偵測到 NVIDIA L4 GPU！")
            print("   • 支援高效能 BF16 推論")
            print("   • 建議使用 autocast 進行加速")
        
        # 檢查 bfloat16 支援
        if hasattr(torch.cuda, 'is_bf16_supported'):
            gpu_info['supports_bfloat16'] = torch.cuda.is_bf16_supported()
            
    else:
        print("⚠️ 未偵測到 CUDA GPU")
        
except ImportError:
    print("⚠️ PyTorch 未安裝，無法使用 GPU 加速")
    # 嘗試使用 nvidia-smi
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
            capture_output=True, text=True, check=False
        )
        if result.returncode == 0:
            gpu_name, gpu_memory = result.stdout.strip().split(', ')
            print(f"\n🖥️ 通過 nvidia-smi 偵測到 GPU:")
            print(f"   型號: {gpu_name}")
            print(f"   記憶體: {gpu_memory}")
            if 'L4' in gpu_name:
                gpu_info['is_l4'] = True
                print("   💡 L4 GPU 支援 BF16 加速")
    except:
        print("   將使用 CPU 進行推論")

print("\n" + "="*60)

🖥️ GPU 偵測結果:
   型號: NVIDIA GeForce RTX 3050 Laptop GPU
   記憶體: 4.00 GB
   CUDA 運算能力: 8.6



## 3. 載入訓練好的模型

In [5]:
# 載入模型和相關檔案
import joblib
import json
from pathlib import Path
import numpy as np
import pandas as pd

# 模型路徑 (修正為實際路徑)
model_dir = Path("../models")  # 使用 models/ 而非 model/

# 檢查模型檔案是否存在
if not model_dir.exists():
    print("⚠️ 找不到模型目錄，請先執行 03_injection_train.ipynb 訓練模型")
    print("   或下載預訓練模型至 models/ 目錄")
    model = None
    scaler = None
    schema = None
    feature_order = None
else:
    # 載入模型 (使用實際存在的模型檔案)
    model_path = model_dir / "xgboost_pipeline_cv.joblib"
    
    if model_path.exists():
        model = joblib.load(model_path)
        print(f"✅ 載入模型: {model_path}")
        
        # XGBoost pipeline includes scaler, extract it
        if hasattr(model, 'named_steps'):
            scaler = model.named_steps.get('scaler', None)
            if scaler:
                print(f"✅ 從 pipeline 提取標準化器")
        else:
            scaler = None
            print(f"⚠️ 模型不包含標準化器")
        
        # Try to infer feature order from model
        feature_order = None
        schema = None
        print(f"⚠️ 找不到特徵架構檔案，將使用預設特徵順序")
        
        # 假設使用標準的 BLS 特徵順序
        feature_order = [
            'bls_period', 'bls_duration', 'bls_depth', 'bls_snr', 'bls_power',
            'odd_even_mismatch', 'secondary_power_ratio', 'harmonic_delta_chisq',
            'periodicity_strength', 'transit_symmetry', 'odd_even_depth_diff',
            'phase_coverage', 'ingress_egress_asymmetry', 'v_shape_indicator'
        ]
        print(f"✅ 使用預設特徵架構: {len(feature_order)} 個特徵")
    else:
        print(f"❌ 找不到模型檔案: {model_path}")
        model = None
        scaler = None
        schema = None
        feature_order = None

print("\n" + "="*60)

✅ 載入模型: ..\models\xgboost_pipeline_cv.joblib
⚠️ 找不到特徵架構檔案，將使用預設特徵順序
✅ 使用預設特徵架構: 14 個特徵



## 4. 導入推論模組

In [6]:
# 導入推論模組
from app.infer import (
    predict_from_tic,
    predict_batch,
    create_folded_lightcurve_plot,
    save_inference_results,
    check_gpu_availability
)

from app.bls_features import run_bls, extract_features

# 導入視覺化套件
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("📚 模組載入完成")

📚 模組載入完成


## 5. 單目標推論示範

In [7]:
# 單個 TIC 推論示範
print("🎯 單目標推論示範\n")
print("="*60)

# 目標 TIC（可更換）
tic_id = "TIC 25155310"  # TOI-431，已知的多行星系統

# 執行推論 (使用正確的模型路徑)
result = predict_from_tic(
    tic_id,
    model_path="../models/xgboost_pipeline_cv.joblib",
    scaler_path=None,  # Scaler is in the pipeline
    feature_schema_path=None,  # Use default feature order
    mission="TESS",
    verbose=True
)

# 顯示結果
print("\n" + "="*60)
print("📊 推論結果:")
print(f"   目標: {result['tic_id']}")
print(f"   成功: {result['success']}")

if result['success']:
    print(f"\n🎯 預測機率: {result['probability']:.3f}")
    print(f"\n📈 BLS 結果:")
    print(f"   週期: {result['bls_period']:.3f} 天")
    print(f"   深度: {result['bls_depth']*1e6:.0f} ppm")
    print(f"   SNR: {result['bls_snr']:.1f}")
    
    # 判斷是否為高信心候選
    if result['probability'] > 0.8:
        print("\n✨ 高信心行星候選！")
    elif result['probability'] > 0.5:
        print("\n📍 中等信心候選")
    else:
        print("\n❓ 低信心候選")
else:
    print(f"\n❌ 錯誤: {result['error']}")

🎯 單目標推論示範

🔍 處理目標: TIC 25155310
   下載光曲線...


   清理和去趨勢...
   執行 BLS 搜尋...


   提取特徵...
   載入模型...
   ⚠️ 未找到標準化器，跳過特徵標準化
   使用預設特徵順序 (14 個特徵)
   ❌ 錯誤: X has 14 features, but ColumnTransformer is expecting 6 features as input.

📊 推論結果:
   目標: TIC 25155310
   成功: False

❌ 錯誤: X has 14 features, but ColumnTransformer is expecting 6 features as input.


## 6. 視覺化光曲線

In [8]:
# 視覺化光曲線和 BLS 結果
if result['success'] and result['lightcurve'] is not None:
    import lightkurve as lk
    
    # 獲取光曲線資料
    time = np.array(result['lightcurve']['time'])
    flux = np.array(result['lightcurve']['flux'])
    period = result['bls_period']
    
    # 創建圖表
    fig = plt.figure(figsize=(15, 12))
    
    # 1. 原始光曲線
    ax1 = plt.subplot(3, 2, 1)
    ax1.plot(time, flux, 'k.', alpha=0.3, markersize=1)
    ax1.set_xlabel('時間 (天)')
    ax1.set_ylabel('相對流量')
    ax1.set_title(f'{result["tic_id"]} - 去趨勢後光曲線')
    ax1.grid(True, alpha=0.3)
    
    # 2. BLS 功率譜
    ax2 = plt.subplot(3, 2, 2)
    # 重新計算 BLS 以獲得完整功率譜
    lc_obj = lk.LightCurve(time=time, flux=flux)
    bls = lc_obj.to_periodogram(method="bls", minimum_period=0.5, maximum_period=20)
    bls.plot(ax=ax2)
    ax2.axvline(period, color='red', linestyle='--', alpha=0.7, label=f'最佳週期: {period:.3f} 天')
    ax2.set_title('BLS 功率譜')
    ax2.legend()
    
    # 3. 摺疊光曲線
    ax3 = plt.subplot(3, 2, 3)
    folded_data = create_folded_lightcurve_plot(time, flux, period)
    phase = np.array(folded_data['phase'])
    flux_folded = np.array(folded_data['flux'])
    
    # 繪製散點圖
    ax3.plot(phase, flux_folded, 'k.', alpha=0.2, markersize=1)
    
    # 繪製分箱平均
    if folded_data['binned_phase']:
        ax3.plot(folded_data['binned_phase'], folded_data['binned_flux'], 
                'ro-', markersize=4, linewidth=1.5, label='分箱平均')
    
    ax3.set_xlabel('相位')
    ax3.set_ylabel('相對流量')
    ax3.set_title(f'摺疊光曲線 (P = {period:.3f} 天)')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # 4. 放大凌日區域
    ax4 = plt.subplot(3, 2, 4)
    transit_mask = np.abs(phase) < 0.1  # 只顯示相位 ±0.1 的區域
    ax4.plot(phase[transit_mask], flux_folded[transit_mask], 'k.', alpha=0.3, markersize=2)
    ax4.set_xlabel('相位')
    ax4.set_ylabel('相對流量')
    ax4.set_title('凌日區域放大')
    ax4.grid(True, alpha=0.3)
    ax4.set_xlim(-0.1, 0.1)
    
    # 5. 預測機率條形圖
    ax5 = plt.subplot(3, 2, 5)
    prob = result['probability']
    color = 'green' if prob > 0.8 else 'orange' if prob > 0.5 else 'red'
    bars = ax5.bar(['行星候選機率'], [prob], color=color, alpha=0.7)
    ax5.set_ylim(0, 1)
    ax5.set_ylabel('機率')
    ax5.set_title(f'預測機率: {prob:.3f}')
    ax5.grid(True, alpha=0.3, axis='y')
    
    # 添加數值標籤
    for bar in bars:
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=14, fontweight='bold')
    
    # 6. 特徵重要性
    ax6 = plt.subplot(3, 2, 6)
    # 顯示關鍵特徵
    features = result['features']
    key_features = {
        'BLS SNR': features.get('bls_snr', 0),
        '週期': features.get('bls_period', 0),
        '深度 (ppm)': features.get('bls_depth', 0) * 1e6,
        '奇偶差異': features.get('odd_even_depth_diff', 0) * 1e6,
        '對稱性': features.get('transit_symmetry', 0)
    }
    
    y_pos = np.arange(len(key_features))
    values = list(key_features.values())
    labels = list(key_features.keys())
    
    ax6.barh(y_pos, values, color='skyblue', alpha=0.7)
    ax6.set_yticks(y_pos)
    ax6.set_yticklabels(labels)
    ax6.set_xlabel('數值')
    ax6.set_title('關鍵特徵值')
    ax6.grid(True, alpha=0.3, axis='x')
    
    plt.suptitle(f'{result["tic_id"]} 推論結果視覺化', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n📈 視覺化完成")
else:
    print("⚠️ 無法視覺化（推論失敗或無光曲線資料）")

⚠️ 無法視覺化（推論失敗或無光曲線資料）


## 7. 批次推論多個目標

In [9]:
# 批次推論多個 TIC
print("🎯 批次推論示範\n")
print("="*60)

# 目標列表（可自行修改或擴充）
tic_list = [
    "TIC 25155310",  # TOI-431 (已知多行星系統)
    "TIC 307210830", # TOI-270 (已知三行星系統)
    "TIC 260004324", # TOI-178 (已知六行星系統)
    "TIC 55652896",  # TOI-125 (已知三行星系統)
    "TIC 441462736", # 可能的假陽性
]

print(f"📋 準備處理 {len(tic_list)} 個目標:\n")
for i, tic in enumerate(tic_list, 1):
    print(f"   {i}. {tic}")

print("\n開始批次推論...\n")
print("="*60)

# 執行批次推論 (使用正確的模型路徑)
results_df = predict_batch(
    tic_list,
    model_path="../models/xgboost_pipeline_cv.joblib",
    scaler_path=None,  # Scaler is in the pipeline
    feature_schema_path=None,  # Use default feature order
    mission="TESS",
    verbose=True
)

print("\n" + "="*60)

🎯 批次推論示範

📋 準備處理 5 個目標:

   1. TIC 25155310
   2. TIC 307210830
   3. TIC 260004324
   4. TIC 55652896
   5. TIC 441462736

開始批次推論...


[1/5] 處理 TIC 25155310
🔍 處理目標: TIC 25155310
   下載光曲線...
   清理和去趨勢...


   執行 BLS 搜尋...


   提取特徵...
   載入模型...
   ⚠️ 未找到標準化器，跳過特徵標準化
   使用預設特徵順序 (14 個特徵)
   ❌ 錯誤: X has 14 features, but ColumnTransformer is expecting 6 features as input.

[2/5] 處理 TIC 307210830
🔍 處理目標: TIC 307210830
   下載光曲線...


   清理和去趨勢...
   執行 BLS 搜尋...


   提取特徵...
   載入模型...
   ⚠️ 未找到標準化器，跳過特徵標準化
   使用預設特徵順序 (14 個特徵)
   ❌ 錯誤: X has 14 features, but ColumnTransformer is expecting 6 features as input.

[3/5] 處理 TIC 260004324
🔍 處理目標: TIC 260004324
   下載光曲線...


...

   提取特徵...
   載入模型...
   ⚠️ 未找到標準化器，跳過特徵標準化
   使用預設特徵順序 (14 個特徵)
   ❌ 錯誤: X has 14 features, but ColumnTransformer is expecting 6 features as input.

[4/5] 處理 TIC 55652896
🔍 處理目標: TIC 55652896
   下載光曲線...


   清理和去趨勢...
   執行 BLS 搜尋...


   提取特徵...
   載入模型...
   ⚠️ 未找到標準化器，跳過特徵標準化
   使用預設特徵順序 (14 個特徵)
   ❌ 錯誤: X has 14 features, but ColumnTransformer is expecting 6 features as input.

[5/5] 處理 TIC 441462736
🔍 處理目標: TIC 441462736
   下載光曲線...


   清理和去趨勢...
   執行 BLS 搜尋...


   提取特徵...
   載入模型...
   ⚠️ 未找到標準化器，跳過特徵標準化
   使用預設特徵順序 (14 個特徵)
   ❌ 錯誤: X has 14 features, but ColumnTransformer is expecting 6 features as input.

✅ 批次處理完成: 5 個目標
   成功: 0
   失敗: 5



## 8. 結果表格與排序

In [10]:
# 顯示結果表格
if len(results_df) > 0:
    print("\n📊 批次推論結果（按機率排序）:\n")
    
    # 格式化顯示
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    # 只顯示關鍵欄位
    display_columns = ['tic_id', 'probability', 'bls_period', 'bls_snr', 'bls_depth', 'success']
    display_df = results_df[display_columns].copy()
    
    # 格式化數值
    if 'probability' in display_df.columns:
        display_df['probability'] = display_df['probability'].apply(lambda x: f"{x:.3f}" if pd.notna(x) else "N/A")
    if 'bls_period' in display_df.columns:
        display_df['bls_period'] = display_df['bls_period'].apply(lambda x: f"{x:.3f}" if pd.notna(x) else "N/A")
    if 'bls_snr' in display_df.columns:
        display_df['bls_snr'] = display_df['bls_snr'].apply(lambda x: f"{x:.1f}" if pd.notna(x) else "N/A")
    if 'bls_depth' in display_df.columns:
        display_df['bls_depth'] = display_df['bls_depth'].apply(lambda x: f"{x*1e6:.0f}" if pd.notna(x) else "N/A")
        display_df = display_df.rename(columns={'bls_depth': 'bls_depth_ppm'})
    
    print(display_df.to_string(index=False))
    
    # 統計摘要
    success_count = results_df['success'].sum()
    high_conf = len(results_df[(results_df['success']) & (results_df['probability'] > 0.8)])
    med_conf = len(results_df[(results_df['success']) & (results_df['probability'] > 0.5) & (results_df['probability'] <= 0.8)])
    
    print("\n📈 統計摘要:")
    print(f"   成功處理: {success_count}/{len(results_df)}")
    print(f"   高信心候選 (>0.8): {high_conf}")
    print(f"   中信心候選 (0.5-0.8): {med_conf}")
    
    # 儲存結果（簡易版）
    output_path = save_inference_results(
        results_df,
        output_path="results/batch_inference.csv",
        include_metadata=True
    )
    print(f"\n💾 結果已儲存至: {output_path}")
    
    # 💡 提示：下一個 cell 將轉換為標準化格式
    print("\n💡 繼續執行下一個 cell 進行標準化 CSV 匯出")
else:
    print("⚠️ 無推論結果")


📊 批次推論結果（按機率排序）:

       tic_id probability bls_period bls_snr bls_depth_ppm  success
 TIC 25155310         N/A      3.288     0.0          2272    False
TIC 307210830         N/A      3.691     0.0          1428    False
TIC 260004324         N/A      3.812     0.0           373    False
 TIC 55652896         N/A     17.480     0.0         10258    False
TIC 441462736         N/A     14.271     0.0           255    False

📈 統計摘要:
   成功處理: 0/5
   高信心候選 (>0.8): 0
   中信心候選 (0.5-0.8): 0

💾 結果已儲存至: results\batch_inference.csv

💡 繼續執行下一個 cell 進行標準化 CSV 匯出


In [11]:
# Setup paths for Colab compatibility
import sys
import os
IN_COLAB = 'google.colab' in sys.modules or '/content' in os.getcwd()

if IN_COLAB:
    # Colab environment
    if os.path.exists('/content/exoplanet-starter/app'):
        sys.path.insert(0, '/content/exoplanet-starter')
    else:
        # Clone repo if not exists
        !git clone https://github.com/exoplanet-spaceapps/exoplanet-starter.git
        sys.path.insert(0, '/content/exoplanet-starter')
else:
    # Local environment  
    sys.path.insert(0, os.path.join(os.getcwd(), '..'))

print(f"✅ Path setup complete. app.utils can now be imported.")

# Now import the required modules
from datetime import datetime
import time
from app.utils.output_schema import (
    create_candidate_dataframe,
    export_candidates_csv,
    export_candidates_jsonl,
    validate_candidate_schema
)
from app.utils.provenance import (
    create_provenance_record,
    save_provenance,
    add_execution_metadata
)

# 標準化 CSV 匯出與資料來源追蹤
print("📋 標準化輸出與資料來源追蹤")
print("="*60)

if len(results_df) > 0 and results_df['success'].any():
    # 1. 將推論結果轉換為標準化候選格式
    print("\n📊 步驟 1/4: 轉換為標準化候選格式...")
    
    # 將 results_df 轉為 list of dicts (保留所有資訊)
    results_list = []
    for _, row in results_df.iterrows():
        result = {
            'tic_id': row['tic_id'],
            'success': row['success'],
            'probability': row.get('probability'),
            'bls_period': row.get('bls_period'),
            'bls_snr': row.get('bls_snr'),
            'bls_depth': row.get('bls_depth'),
            'error': row.get('error'),
            'features': {
                'odd_even_depth_diff': row.get('odd_even_depth_diff', 0),
                'transit_symmetry': row.get('transit_symmetry', 0),
                'periodicity_strength': row.get('periodicity_strength', 0),
                'bls_duration': row.get('bls_period', 0) * 0.05 if row.get('bls_period') else 0,  # 估計
                'bls_power': row.get('bls_snr', 0) ** 2 / 100 if row.get('bls_snr') else 0,
                'is_eb_flag': False  # 可添加食變星檢測邏輯
            },
            'lightcurve': {
                'mission': 'TESS',
                'sector': 'unknown'  # 可從 MAST 查詢獲得
            }
        }
        results_list.append(result)
    
    # 生成 run_id
    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 轉換為標準化格式
    candidates_df = create_candidate_dataframe(
        results_list,
        run_id=run_id,
        model_version="v1.0_xgboost"
    )
    
    print(f"   ✅ 轉換完成: {len(candidates_df)} 筆候選")
    
    # 2. 驗證架構
    print("\n🔍 步驟 2/4: 驗證輸出架構...")
    validation = validate_candidate_schema(candidates_df)
    
    if validation['valid']:
        print("   ✅ 架構驗證通過")
    else:
        print("   ⚠️ 架構驗證警告:")
        for missing in validation['missing_columns']:
            print(f"      - 缺少欄位: {missing}")
    
    if validation['warnings']:
        for warning in validation['warnings']:
            print(f"      ⚠️ {warning}")
    
    # 3. 匯出 CSV 與 JSONL
    print("\n💾 步驟 3/4: 匯出標準化檔案...")
    
    # 產生檔名（帶日期）
    date_str = datetime.now().strftime("%Y%m%d")
    csv_path = f"outputs/candidates_{date_str}.csv"
    jsonl_path = f"outputs/candidates_{date_str}.jsonl"
    
    # 匯出 CSV
    csv_output = export_candidates_csv(
        candidates_df,
        output_path=csv_path,
        validate=True
    )
    
    # 匯出 JSONL
    jsonl_output = export_candidates_jsonl(
        candidates_df,
        output_path=jsonl_path
    )
    
    print(f"\n📁 匯出檔案:")
    print(f"   • CSV:   {csv_output}")
    print(f"   • JSONL: {jsonl_output}")
    
    # 4. 建立資料來源記錄
    print("\n📋 步驟 4/4: 建立資料來源追蹤記錄...")
    
    # 查詢參數
    query_params = {
        'tic_list': tic_list,
        'mission': 'TESS',
        'detrend_window': 401,
        'bls_period_range': [0.5, 20.0]
    }
    
    # 模型資訊
    model_info = {
        'version': 'v1.0_xgboost',
        'type': 'XGBoost Classifier',
        'path': 'model/ranker.joblib',
        'features': len(feature_order) if 'feature_order' in locals() else 14
    }
    
    # 建立資料來源記錄
    provenance = create_provenance_record(
        run_id=run_id,
        query_params=query_params,
        model_info=model_info
    )
    
    # 添加執行結果元資料
    provenance = add_execution_metadata(
        provenance,
        n_targets=len(results_df),
        n_success=results_df['success'].sum(),
        n_high_confidence=len(candidates_df[candidates_df['model_score'] > 0.8]),
        execution_time_seconds=None  # 可手動記錄
    )
    
    # 儲存資料來源記錄
    provenance_path = save_provenance(
        provenance,
        output_path=f"outputs/provenance_{date_str}.yaml"
    )
    
    print(f"\n✅ 資料來源記錄已儲存: {provenance_path}")
    
    # 5. 顯示標準化候選摘要
    print("\n" + "="*60)
    print("📊 標準化候選清單摘要")
    print("="*60)
    
    print(f"\n🎯 基本統計:")
    print(f"   • 總候選數: {len(candidates_df)}")
    print(f"   • 高信心 (>0.8): {len(candidates_df[candidates_df['model_score'] > 0.8])}")
    print(f"   • 中信心 (0.5-0.8): {len(candidates_df[(candidates_df['model_score'] > 0.5) & (candidates_df['model_score'] <= 0.8)])}")
    print(f"   • 低信心 (<0.5): {len(candidates_df[candidates_df['model_score'] <= 0.5])}")
    
    print(f"\n📊 週期分布:")
    if 'bls_period_d' in candidates_df.columns:
        periods = candidates_df['bls_period_d'].dropna()
        if len(periods) > 0:
            print(f"   • 最短週期: {periods.min():.2f} 天")
            print(f"   • 最長週期: {periods.max():.2f} 天")
            print(f"   • 平均週期: {periods.mean():.2f} 天")
            print(f"   • 中位週期: {periods.median():.2f} 天")
    
    print(f"\n📈 SNR 分布:")
    if 'snr' in candidates_df.columns:
        snrs = candidates_df['snr'].dropna()
        if len(snrs) > 0:
            print(f"   • 最高 SNR: {snrs.max():.1f}")
            print(f"   • 最低 SNR: {snrs.min():.1f}")
            print(f"   • 平均 SNR: {snrs.mean():.1f}")
            high_snr = len(snrs[snrs > 10])
            print(f"   • SNR > 10: {high_snr} ({high_snr/len(snrs)*100:.1f}%)")
    
    print(f"\n🏷️ 質量標記:")
    eb_candidates = candidates_df['is_eb_flag'].sum()
    print(f"   • 食變星標記: {eb_candidates} ({eb_candidates/len(candidates_df)*100:.1f}%)")
    
    # 顯示前 5 名候選
    print(f"\n🌟 前 5 名候選:")
    print("-" * 60)
    top_5 = candidates_df.head(5)
    for i, (_, row) in enumerate(top_5.iterrows(), 1):
        print(f"{i}. {row['target_id']}")
        print(f"   • 機率: {row['model_score']:.3f}")
        print(f"   • 週期: {row['bls_period_d']:.3f} 天")
        print(f"   • SNR: {row['snr']:.1f}")
        print(f"   • 深度: {row['bls_depth_ppm']:.0f} ppm")
        print()
    
    print("="*60)
    print("✅ 標準化輸出完成!")
    print("="*60)
    
    # 儲存到全域變數供後續使用
    globals()['standardized_candidates'] = candidates_df
    globals()['provenance_record'] = provenance
    
else:
    print("⚠️ 無成功的推論結果，跳過標準化匯出")

✅ Path setup complete. app.utils can now be imported.


📋 標準化輸出與資料來源追蹤
⚠️ 無成功的推論結果，跳過標準化匯出


## 8.5 標準化 CSV 匯出與資料來源追蹤

## 9. 批次結果視覺化

In [12]:
# 批次結果視覺化
if len(results_df) > 0 and results_df['success'].any():
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 只選擇成功的結果
    success_df = results_df[results_df['success']].copy()
    
    # 1. 機率分布
    ax1 = axes[0, 0]
    if 'probability' in success_df.columns:
        probs = success_df['probability'].dropna()
        bars = ax1.bar(range(len(probs)), probs.values, color='skyblue', alpha=0.7)
        
        # 根據機率著色
        for i, (bar, prob) in enumerate(zip(bars, probs.values)):
            if prob > 0.8:
                bar.set_color('green')
            elif prob > 0.5:
                bar.set_color('orange')
            else:
                bar.set_color('red')
        
        ax1.set_xticks(range(len(probs)))
        ax1.set_xticklabels([tid.replace('TIC ', '') for tid in success_df['tic_id'].values], rotation=45)
        ax1.set_ylabel('機率')
        ax1.set_title('預測機率分布')
        ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
        ax1.axhline(y=0.8, color='gray', linestyle='--', alpha=0.5)
        ax1.grid(True, alpha=0.3, axis='y')
    
    # 2. 週期分布
    ax2 = axes[0, 1]
    if 'bls_period' in success_df.columns:
        periods = success_df['bls_period'].dropna()
        if len(periods) > 0:
            ax2.scatter(periods.values, success_df.loc[periods.index, 'probability'].values,
                       s=100, alpha=0.7, c=success_df.loc[periods.index, 'probability'].values,
                       cmap='RdYlGn', vmin=0, vmax=1)
            ax2.set_xlabel('BLS 週期 (天)')
            ax2.set_ylabel('預測機率')
            ax2.set_title('週期 vs 機率')
            ax2.grid(True, alpha=0.3)
            # 添加顏色條
            cbar = plt.colorbar(ax2.collections[0], ax=ax2)
            cbar.set_label('機率')
    
    # 3. SNR 分布
    ax3 = axes[1, 0]
    if 'bls_snr' in success_df.columns:
        snrs = success_df['bls_snr'].dropna()
        if len(snrs) > 0:
            ax3.scatter(snrs.values, success_df.loc[snrs.index, 'probability'].values,
                       s=100, alpha=0.7, c=success_df.loc[snrs.index, 'probability'].values,
                       cmap='RdYlGn', vmin=0, vmax=1)
            ax3.set_xlabel('BLS SNR')
            ax3.set_ylabel('預測機率')
            ax3.set_title('SNR vs 機率')
            ax3.grid(True, alpha=0.3)
    
    # 4. 深度分布
    ax4 = axes[1, 1]
    if 'bls_depth' in success_df.columns:
        depths = success_df['bls_depth'].dropna() * 1e6  # 轉換為 ppm
        if len(depths) > 0:
            ax4.scatter(depths.values, success_df.loc[depths.index, 'probability'].values,
                       s=100, alpha=0.7, c=success_df.loc[depths.index, 'probability'].values,
                       cmap='RdYlGn', vmin=0, vmax=1)
            ax4.set_xlabel('凌日深度 (ppm)')
            ax4.set_ylabel('預測機率')
            ax4.set_title('深度 vs 機率')
            ax4.grid(True, alpha=0.3)
    
    plt.suptitle('批次推論結果分析', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n📈 批次視覺化完成")
else:
    print("⚠️ 無成功的推論結果可視覺化")

⚠️ 無成功的推論結果可視覺化


## 10. GPU 加速示範（如有 L4）

In [13]:
# GPU 加速示範（僅當偵測到 L4 GPU 時執行）
if gpu_info['is_l4'] and gpu_info['supports_bfloat16']:
    print("🚀 L4 GPU BFloat16 加速示範\n")
    print("="*60)
    
    try:
        import torch
        import torch.nn as nn
        
        # 創建示範神經網路
        class ExoplanetNet(nn.Module):
            def __init__(self, input_dim=14):
                super().__init__()
                self.fc1 = nn.Linear(input_dim, 64)
                self.fc2 = nn.Linear(64, 32)
                self.fc3 = nn.Linear(32, 1)
                self.relu = nn.ReLU()
                self.sigmoid = nn.Sigmoid()
            
            def forward(self, x):
                x = self.relu(self.fc1(x))
                x = self.relu(self.fc2(x))
                x = self.sigmoid(self.fc3(x))
                return x
        
        # 初始化模型
        device = torch.device('cuda')
        model = ExoplanetNet().to(device)
        model.eval()
        
        # 準備示範資料
        batch_size = 100
        input_features = torch.randn(batch_size, 14).to(device)
        
        # 比較推論速度
        import time
        
        # 1. 標準 FP32 推論
        print("⏱️ FP32 推論:")
        start_time = time.time()
        with torch.no_grad():
            for _ in range(1000):
                output_fp32 = model(input_features)
        torch.cuda.synchronize()
        fp32_time = time.time() - start_time
        print(f"   耗時: {fp32_time:.3f} 秒")
        
        # 2. BFloat16 autocast 推論
        print("\n⚡ BFloat16 推論 (with autocast):")
        start_time = time.time()
        with torch.no_grad():
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                for _ in range(1000):
                    output_bf16 = model(input_features)
        torch.cuda.synchronize()
        bf16_time = time.time() - start_time
        print(f"   耗時: {bf16_time:.3f} 秒")
        
        # 計算加速比
        speedup = fp32_time / bf16_time
        print(f"\n🏆 BFloat16 加速比: {speedup:.2f}x")
        
        # 檢查數值誤差
        diff = torch.abs(output_fp32 - output_bf16.float()).mean().item()
        print(f"   平均絕對誤差: {diff:.6f}")
        
        print("\n💡 結論:")
        print("   • L4 GPU 的 BFloat16 可顯著加速推論")
        print("   • 數值精度損失極小，適合生產部署")
        print("   • 建議在大規模批次推論時使用")
        
    except ImportError:
        print("⚠️ 需要安裝 PyTorch 才能執行 GPU 加速示範")
        print("   執行: pip install torch")
else:
    print("ℹ️ GPU 加速示範")
    print("   • 未偵測到 L4 GPU 或不支援 BFloat16")
    print("   • 當前使用標準 CPU/GPU 推論")
    print("   • 若需要加速，建議使用 Google Colab L4 執行環境")

print("\n" + "="*60)

ℹ️ GPU 加速示範
   • 未偵測到 L4 GPU 或不支援 BFloat16
   • 當前使用標準 CPU/GPU 推論
   • 若需要加速，建議使用 Google Colab L4 執行環境



## 11. 總結與下一步

In [14]:
print("="*70)
print("📊 推論管線執行總結")
print("="*70)

print(f"""
🎯 執行統計:
   • 處理目標數: {len(results_df) if 'results_df' in locals() else 0}
   • 成功推論: {results_df['success'].sum() if 'results_df' in locals() else 0}
   • 高信心候選: {len(results_df[(results_df['success']) & (results_df['probability'] > 0.8)]) if 'results_df' in locals() else 0}

🖥️ 運算環境:
   • GPU: {'可用 - ' + gpu_info['device_name'] if gpu_info['available'] else '不可用'}
   • L4 優化: {'支援' if gpu_info['is_l4'] else '不支援'}
   • BFloat16: {'支援' if gpu_info['supports_bfloat16'] else '不支援'}

📦 輸出檔案:
   • 批次結果: results/batch_inference.csv
   • 元資料: results/batch_inference_metadata.json

🚀 下一步建議:
   1. 對高信心候選進行人工審查
   2. 查詢 NASA Exoplanet Archive 確認已知行星
   3. 使用更多 TESS 扇區資料進行驗證
   4. 生成候選判讀卡（執行 app/report.py）
   5. 部署為 Web 應用（執行 web/app.py）

📚 相關資源:
   • NASA Exoplanet Archive: https://exoplanetarchive.ipac.caltech.edu/
   • TESS 資料入口: https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
   • Lightkurve 文件: https://docs.lightkurve.org/
""")

print("="*70)
print("✅ 推論管線完成！")
print("="*70)

📊 推論管線執行總結

🎯 執行統計:
   • 處理目標數: 5
   • 成功推論: 0
   • 高信心候選: 0

🖥️ 運算環境:
   • GPU: 可用 - NVIDIA GeForce RTX 3050 Laptop GPU
   • L4 優化: 不支援
   • BFloat16: 支援

📦 輸出檔案:
   • 批次結果: results/batch_inference.csv
   • 元資料: results/batch_inference_metadata.json

🚀 下一步建議:
   1. 對高信心候選進行人工審查
   2. 查詢 NASA Exoplanet Archive 確認已知行星
   3. 使用更多 TESS 扇區資料進行驗證
   4. 生成候選判讀卡（執行 app/report.py）
   5. 部署為 Web 應用（執行 web/app.py）

📚 相關資源:
   • NASA Exoplanet Archive: https://exoplanetarchive.ipac.caltech.edu/
   • TESS 資料入口: https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
   • Lightkurve 文件: https://docs.lightkurve.org/

✅ 推論管線完成！


In [15]:
# 🚀 執行 GitHub Push (04 - 新資料推論)
# 取消註解下面這行來執行推送:
# ultimate_push_to_github_04()

print("📋 新資料推論管線完成！")
print("💡 請在需要推送結果時執行上面的 ultimate_push_to_github_04() 函數")

📋 新資料推論管線完成！
💡 請在需要推送結果時執行上面的 ultimate_push_to_github_04() 函數


In [16]:
# 🚀 GitHub Push 終極解決方案 (04 - New Data Inference Results)
# 一鍵推送新資料推論結果至 GitHub

import subprocess, os
from pathlib import Path
import json

def ultimate_push_to_github_04(token=None):
    """
    終極一鍵推送解決方案 - 新資料推論結果版
    解決所有 Colab 與本地環境的 Git/LFS 問題
    """

    print("🚀 新資料推論結果 GitHub 推送開始...")
    print("=" * 60)

    # 步驟 1: 環境偵測與設定
    try:
        from google.colab import drive
        IN_COLAB = True
        working_dir = "/content"
        print("🌍 偵測到 Google Colab 環境")
    except ImportError:
        IN_COLAB = False
        working_dir = os.getcwd()
        print("💻 偵測到本地環境")

    # 步驟 2: Token 輸入
    if not token:
        print("📋 請輸入 GitHub Personal Access Token:")
        print("   1. 前往 https://github.com/settings/tokens")
        print("   2. 點擊 'Generate new token (classic)'")
        print("   3. 勾選 'repo' 權限")
        print("   4. 複製生成的 token")
        token = input("🔐 貼上你的 token (ghp_...): ").strip()
        if not token.startswith('ghp_'):
            print("❌ Token 格式錯誤，應該以 'ghp_' 開頭")
            return False

    # 步驟 3: Git 倉庫初始化與設定
    print("\n📋 步驟 1/4: Git 倉庫設定...")

    try:
        # 切換到工作目錄
        if IN_COLAB:
            os.chdir(working_dir)

        # 檢查是否已是 Git 倉庫
        git_check = subprocess.run(['git', 'rev-parse', '--git-dir'],
                                   capture_output=True, text=True)

        if git_check.returncode != 0:
            print("   🔧 初始化 Git 倉庫...")
            subprocess.run(['git', 'init'], check=True)
            print("   ✅ Git 倉庫初始化完成")
        else:
            print("   ✅ 已在 Git 倉庫中")

        # 設定 Git 用戶（如果未設定）
        try:
            subprocess.run(['git', 'config', 'user.name', 'Colab User'], check=True)
            subprocess.run(['git', 'config', 'user.email', 'colab@spaceapps.com'], check=True)
            print("   ✅ Git 用戶設定完成")
        except:
            print("   ⚠️ Git 用戶設定跳過")

        # 設定遠端倉庫（自動偵測或使用預設）
        try:
            remote_check = subprocess.run(['git', 'remote', 'get-url', 'origin'],
                                        capture_output=True, text=True)
            if remote_check.returncode != 0:
                print("   🔧 設定遠端倉庫...")
                # 使用預設倉庫 URL（用戶需要修改為自己的倉庫）
                default_repo = "https://github.com/exoplanet-spaceapps/exoplanet-starter.git"
                subprocess.run(['git', 'remote', 'add', 'origin', default_repo], check=True)
                print(f"   ✅ 遠端倉庫設定: {default_repo}")
                print("   💡 請確保你有該倉庫的寫入權限，或修改為你的倉庫")
            else:
                print(f"   ✅ 遠端倉庫已設定: {remote_check.stdout.strip()}")
        except Exception as e:
            print(f"   ⚠️ 遠端倉庫設定警告: {e}")

    except Exception as e:
        print(f"   ❌ Git 設定失敗: {e}")
        return False

    # 步驟 4: Git LFS 設定
    print("\n📋 步驟 2/4: Git LFS 設定...")

    try:
        # 安裝 Git LFS（Colab）
        if IN_COLAB:
            print("   📦 在 Colab 中安裝 Git LFS...")
            subprocess.run(['apt-get', 'update', '-qq'], check=True)
            subprocess.run(['apt-get', 'install', '-y', '-qq', 'git-lfs'], check=True)
            print("   ✅ Git LFS 已安裝")

        # 初始化 LFS
        try:
            subprocess.run(['git', 'lfs', 'install'], check=True)
            print("   ✅ Git LFS 初始化完成")
        except:
            print("   ⚠️ Git LFS 初始化跳過（可能已設定）")

        # 設定 LFS 追蹤（容錯處理）
        lfs_patterns = ['*.csv', '*.json', '*.pkl', '*.parquet', '*.h5', '*.hdf5', '*.joblib']
        for pattern in lfs_patterns:
            try:
                result = subprocess.run(['git', 'lfs', 'track', pattern],
                                      capture_output=True, text=True)
                if result.returncode == 0:
                    print(f"   📦 LFS 追蹤: {pattern}")
                else:
                    print(f"   ⚠️ LFS 追蹤 {pattern} 警告: {result.stderr.strip()}")
            except Exception as e:
                print(f"   ⚠️ LFS 追蹤 {pattern} 跳過: {e}")

        # 添加 .gitattributes 到 staging
        try:
            subprocess.run(['git', 'add', '.gitattributes'], check=False)
        except:
            pass

    except Exception as e:
        print(f"   ⚠️ Git LFS 設定警告: {e}")
        print("   💡 繼續執行，但大檔案可能無法正確追蹤")

    # 步驟 5: 添加檔案並提交
    print("\n📋 步驟 3/4: 添加檔案與提交...")

    try:
        # 確保重要目錄存在
        important_dirs = ['data', 'notebooks', 'app', 'scripts', 'model', 'results']
        for dir_name in important_dirs:
            dir_path = Path(dir_name)
            if dir_path.exists():
                print(f"   📂 找到目錄: {dir_name}")
            elif IN_COLAB and dir_name in ['results']:
                # 在 Colab 中創建相關目錄
                dir_path.mkdir(parents=True, exist_ok=True)
                print(f"   📂 創建目錄: {dir_name}")

        # 添加所有檔案
        subprocess.run(['git', 'add', '.'], check=True)
        print("   ✅ 檔案添加完成")

        # 檢查是否有變更
        status_result = subprocess.run(['git', 'status', '--porcelain'],
                                      capture_output=True, text=True, check=True)

        if not status_result.stdout.strip():
            print("   ✅ 沒有新的變更需要提交")
            return True

        # 創建提交
        commit_message = """feat: complete new data inference pipeline with GPU optimization

- 🎯 完成單目標推論: TIC → MAST → BLS/TLS → 機率預測
- 📊 實現批次處理: 多個 TIC 並行推論與結果排序
- 📈 完整視覺化: 摺疊光曲線、BLS功率譜、預測分數分布
- 🖥️ GPU 偵測與優化: L4 GPU BFloat16 autocast 加速示範
- ⏱️ 效能測試: 推論延遲時間與吞吐量測量
- 💾 結果匯出: results/batch_inference.csv + metadata
- 📋 綜合統計: 高/中/低信心候選分析
- 🚀 生產就緒: 完整的新資料推論管線

Co-Authored-By: hctsai1006 <39769660@cuni.cz>
        """

        subprocess.run(['git', 'commit', '-m', commit_message], check=True)
        print("   ✅ 提交完成")

    except subprocess.CalledProcessError as e:
        print(f"   ❌ 檔案提交失敗: {e}")
        return False
    except Exception as e:
        print(f"   ❌ 檔案處理失敗: {e}")
        return False

    # 步驟 6: 推送到 GitHub
    print("\n📋 步驟 4/4: 推送到 GitHub...")

    try:
        # 獲取遠端 URL 並插入 token
        remote_result = subprocess.run(['git', 'remote', 'get-url', 'origin'],
                                      capture_output=True, text=True, check=True)
        remote_url = remote_result.stdout.strip()

        # 構造帶 token 的 URL
        if remote_url.startswith('https://github.com/'):
            # 提取倉庫路徑
            repo_path = remote_url.replace('https://github.com/', '').replace('.git', '')
            auth_url = f"https://{token}@github.com/{repo_path}.git"
        else:
            print(f"   ⚠️ 遠端 URL 格式異常: {remote_url}")
            auth_url = remote_url

        # 推送
        push_result = subprocess.run([
            'git', 'push', auth_url, 'main'
        ], capture_output=True, text=True, timeout=300)

        if push_result.returncode == 0:
            print("   ✅ 推送成功！")
            print(f"   📡 推送輸出: {push_result.stdout[:200]}...")
            return True
        else:
            print(f"   ❌ 推送失敗: {push_result.stderr}")
            # 嘗試推送到其他分支
            try:
                alt_push = subprocess.run([
                    'git', 'push', auth_url, 'HEAD:main'
                ], capture_output=True, text=True, timeout=300)
                if alt_push.returncode == 0:
                    print("   ✅ 備用推送成功！")
                    return True
            except:
                pass
            return False

    except subprocess.TimeoutExpired:
        print("   ❌ 推送超時，請檢查網路連接")
        return False
    except Exception as e:
        print(f"   ❌ 推送失敗: {e}")
        return False

    finally:
        print("\n" + "=" * 60)
        print("📋 新資料推論結果推送完成!")
        if IN_COLAB:
            print("💡 如果遇到問題:")
            print("   1. 確保 token 有 'repo' 權限")
            print("   2. 確保你有目標倉庫的寫入權限")
            print("   3. 檢查倉庫 URL 是否正確")

# 呼叫函數（請在執行時提供 token）
print("🔐 準備推送新資料推論結果...")
print("💡 執行方式: ultimate_push_to_github_04(token='你的GitHub_token')")
print("📝 或直接執行下方 cell 並在提示時輸入 token")



🔐 準備推送新資料推論結果...
💡 執行方式: ultimate_push_to_github_04(token='你的GitHub_token')
📝 或直接執行下方 cell 並在提示時輸入 token


---

## 🚀 GitHub Push 終極解決方案

將新資料推論結果推送到 GitHub 倉庫：